In [1]:
import numpy as np
import pandas as pd
import time

### *Simplex* Function

In [2]:
def max2minsub(A, b, c):

    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    new_A = A.copy()
    new_b = b.copy()
    new_c = c.copy()
    bound = None  # Store the bound to reverse the transformation later
    bound_col = None  # Store the column index of the bound variable

    for i in range(len(b)):
        row = A[i, :]
        nonzero_indices = np.where(row != 0)[0]

        # Case 1: Adjust variables with explicit lower bound (b[i] <= 0, one -1 in row)
        if b[i] <= 0 and len(nonzero_indices) == 1 and row[nonzero_indices[0]] == -1:
            bound_col = nonzero_indices[0]
            bound = -b[i]  # Store the bound value

            # Adjust b only for rows different from the detected one
            for j in range(len(b)):
                new_b[j] -= A[j, bound_col] * bound  # Adjust without multiplying by bound

            # Change the -1 to 1 in the detected row
            new_A[i, bound_col] *= -1

            break  # Stop after processing the first detected constraint

        # Case 2: Flip the sign of an entire column if b[i] == 0 and exactly one -1 exists
        elif b[i] == 0 and len(nonzero_indices) == 1 and row[nonzero_indices[0]] == 1:
            col_idx = nonzero_indices[0]  # Get the column index of -1

            # Flip the signs of the entire column in A
            new_A[:, col_idx] *= -1

            # Flip the sign of the corresponding coefficient in c
            new_c[col_idx] *= -1

    return new_A, new_b, new_c, bound, bound_col


def check_feasible(A, b):
    for i in range(len(b)):
        # Check row and b sign
        row = A[i, :]
        if np.all(row <= 0) and b[i] > 0:
            return False
        if np.all(row >= 0) and b[i] < 0:
            return False
    return True

def simplex(c, A, b, is_max=True, verbose=False, maxIter=100):
    # Convert lists to numpy arrays
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    # Check x_i >= const conditions
    A, b, c, bound, bound_col = max2minsub(A, b, c)

    # Number of variables and slack variables
    num_var = len(c)
    num_slacks, _ = A.shape

    # Extend A and C with slack variables
    A_ext = np.hstack((A, np.eye(num_slacks)))
    c_ext = np.concatenate((c, np.zeros(num_slacks)))

    # Track basic variables
    basic_vars = list(range(num_var, num_var + num_slacks))

    # Changes for minimization
    if is_max:
        c_ext = -c_ext

    # Iteration counter
    iter_count = 0

    # Cost function
    z = 0

    # Check initial feasibility
    if not check_feasible(A_ext, b):
        if verbose:
            print("The system is not feasible in the initial phase.")
        return None, None, False

    # "do-while" loop using while True
    while True:
        iter_count += 1

        # >> 1. Check optimality condition (only after the first iteration)
        if np.min(c_ext) >= 0:
            break  # Stop if optimality is reached

        # >> 2. Pivot Column (most negative coefficient in c_ext)
        col_piv = np.argmin(c_ext)

        # >> 3. Pivot Row (Minimum ratio test)
        ratios = np.full(b.shape, np.inf)  # Initialize ratios as infinity
        for i in range(len(b)):
            if A_ext[i, col_piv] != 0:
                ratios_temp = b[i] / A_ext[i, col_piv]
                if ratios_temp > 0:
                    ratios[i] = ratios_temp

        # If all ratios are infinity, the problem is unbounded
        if np.all(ratios == np.inf):
            if verbose:
                print("The problem is unbounded.")
            return None, None, False

        row_piv = np.argmin(ratios)

        # >> 4. Pivot Element
        element_piv = A_ext[row_piv, col_piv]

        # >> 5. Pivot Row Normalization
        A_ext[row_piv, :] /= element_piv
        b[row_piv] /= element_piv

        # >> 6. Row operations to eliminate pivot column
        for i in range(len(b)):
            if i != row_piv:
                row_coef = A_ext[i, col_piv]
                A_ext[i, :] -= row_coef * A_ext[row_piv, :]
                b[i] -= row_coef * b[row_piv]

        # >> 7. Function row update
        z -= c_ext[col_piv] * b[row_piv]  # Correct update of objective function
        c_ext -= c_ext[col_piv] * A_ext[row_piv, :]

        # Update basic variables
        basic_vars[row_piv] = col_piv

        # Stop if max iterations reached
        if iter_count >= maxIter:
            if verbose:
                print("Maximum number of iterations reached.")
            return None, None, False

    # Final feasibility check
    if not check_feasible(A_ext, b):
        if verbose:
            print("The system is not feasible after optimization.")
        return None, None, False

    # Obtain solution
    x = np.zeros(num_var)
    for i, var in enumerate(basic_vars):
        if var < num_var:
            x[var] = b[i]

    if not is_max:
        z = -z  # Adjust objective function for minimization

    if bound_col is not None:
      x[bound_col] += bound  # Restore the original variable

    return x, z, True



### Branch and Bound

In [3]:
def BB(x_ini, z_ini, A, b, c, best_x, best_z, epsilon=1e-6):
  # >> 1. Select a fractional variable (prioritize the first)
  frac_indices = np.where(np.abs(x_ini - np.round(x_ini)) > epsilon)[0]

  # If no fractional variables remain, update the best solution
  if len(frac_indices) == 0:  # If no fractional variables, update best solution
    if z_ini > best_z:
        return x_ini, z_ini  # Update best solution
    return best_x, best_z  # Keep previous best solution
  idx = frac_indices[0]

  # >> 2. Generate new constraints for branching

  # Lower bound constraint: x_i <= floor(x_i)
  new_row1 = np.zeros(A.shape[1])
  new_row1[idx] = 1
  A1 = np.vstack((A, new_row1))
  b1 = np.append(b, np.floor(x_ini[idx]))

  # Upper bound constraint: x_i >= ceil(x_i) -> -x_i <= -ceil(x_i)
  new_row2 = np.zeros(A.shape[1])
  new_row2[idx] = -1
  A2 = np.vstack((A, new_row2))
  b2 = np.append(b, -np.ceil(x_ini[idx]))

  # 3. Solve using Simplex
  x1, z1, feasible1 = simplex(c, A1, b1)
  x2, z2, feasible2 = simplex(c, A2, b2)

  # 4. Bounding and Recursion
  if feasible1 and z1 > best_z:
      best_x, best_z = BB(x1, z1, A1, b1, c, best_x, best_z, epsilon)
  if feasible2 and z2 > best_z:
      best_x, best_z = BB(x2, z2, A2, b2, c, best_x, best_z, epsilon)

  return best_x, best_z

### Integer Simplex

In [4]:
def IntegerSimplex(c, A, b, maxIter):
    # Convert lists to numpy arrays
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    # 1. Run Simplex method
    x_ini, z_ini, feasible = simplex(c, A, b, maxIter)

    if not feasible:
        return None, None  # No feasible solution

    # 2. Check if the solution is already integer
    if np.all(x_ini == np.floor(x_ini)):
        return x_ini, z_ini

    # 3. Start Branch and Bound
    best_x, best_z = None, -np.inf
    best_x, best_z = BB(x_ini, best_z, A, b, c, best_x, best_z)

    return best_x, best_z  # Return the best integer solution found

### Testing

In [9]:
# Juguetería x = (60,20)
A = [[1, 1],
     [1, 2],
     [0, 1]]
b = [80,
     100,
     40]
c = [2000, 3000]

# Resistencia x = (2,5)
A = [[-5, -8]]
b = [-50]
c = [2, 3]

# Integer x = (0, 33)
A = [[2.5, 1.5],
     [1, 0.5]]
b = [50,
     30]
c = [5, 4]

# Integer x = (25, 0)
A = [[2, 3]]
b = [51]
c = [10, 5]


# Start Time
start_time = time.time()
x, z, feasible = simplex(c, A, b, True, True)

# End time
end_time = time.time()
execution_time = end_time - start_time

print(f'CONTINUOUS SIMPLEX')
print(f'Optimal Solution: {x}')
print(f'Best Value: {z}')
print(f'Execution Time: {execution_time:.9f}')
print(f'Feasible = {feasible} \n')


# Start Time
start_time = time.time()
x, z = IntegerSimplex(c, A, b, 10)

# End time
end_time = time.time()
execution_time = end_time - start_time

print(f'INTEGER SIMPLEX')
print(f'Optimal Solution: {x}')
print(f'Best Value: {z}')
print(f'Execution Time: {execution_time:.9f} \n')


CONTINUOUS SIMPLEX
Optimal Solution: [25.5  0. ]
Best Value: 255.0
Execution Time: 0.001290798
Feasible = True 

INTEGER SIMPLEX
Optimal Solution: [25.  0.]
Best Value: 250.0
Execution Time: 0.020653963 

